In [ ]:
import os
import re
import sys
import glob
import json
import dask
import shutil
import pickle
import hashlib
import skimage
import datetime
import tifffile
import numpy as np
import pandas as pd

import dask.diagnostics
from matplotlib import pyplot as plt

In [ ]:
sys.path.append('..')
from pipeline_process.imaging import image, plate_microscopy_api, utils, viz

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
ess_root = '/gpfsML/ML_group/PlateMicroscopy/'
os.path.isdir(ess_root)

### Parsing all raw TIFF metadata

In [ ]:
api = plate_microscopy_api.PlateMicroscopyAPI(ess_root, '../cache/20191029-ess-on-cap/')

In [ ]:
# api.cache_os_walk()
api.construct_metadata()
api.append_file_info()
api.cache_metadata(overwrite=True)

In [ ]:
len(api.os_walk), api.md.shape, api.md.is_raw.sum()

In [ ]:
d_raw = api.md.loc[api.md.is_raw].copy()
d_raw['exp_id'] = [exp_dir.split('_')[0] for exp_dir in d_raw.exp_dir]
d_raw['plate_num'] = d_raw.plate_num.apply(int)

In [ ]:
dst_root = '/gpfsML/ML_group/PlateMicroscopy-metadata/'

tasks = []
for ind, row in d_raw.iterrows():
    task = dask.delayed(api.parse_raw_file(row, src_root=api.root_dir, dst_root=dst_root))
    tasks.append(task)

In [ ]:
with ProgressBar():
    dask.compute(*tasks)